# User Documetation

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# use a better colormap and don't interpolate the pixels
matplotlib.rc('image', cmap='inferno')
matplotlib.rc('image', interpolation='none')

import scarlet
import scarlet.display
import scarlet.constraints as sc
from scarlet.config import Config

# Load the sample images
data = np.load("../data/test_sim/data.npz")
images = data["images"]
filters = data["filters"]

# Load the detection catalog
from astropy.table import Table as ApTable
catalog = ApTable.read("../data/test_sim/true_catalog.fits")
bg_rms = np.array([20]*len(images))
# Set the arcsinh color scaling object
asinh = scarlet.display.Asinh(img=images, Q=50)

# Display the sources
def display_sources(sources, subset=None, combine=False, show_sed=True):
    """Display the data and model for all sources in a blend
    
    This convenience function is used to display all (or a subset) of
    the sources and (optionally) their SED's.
    """
    if subset is None:
        # Show all sources in the blend
        subset = range(len(sources))
    for m in subset:
        # Load the model for the source
        src = sources[m]
        model = src.get_model(combine=combine)
        # Since a model can be generated for each component in a source,
        # or all components can be combined into a single model,
        # reshape the model if there is only a single component
        if len(model.shape)==4:
            components = model.shape[0]
        else:
            model = np.expand_dims(model, axis=0)

        # Select the image patch the overlaps with the source and convert it to an RGB image
        img_rgb = scarlet.display.img_to_rgb(images[src.bb], filter_indices=[3,2,1], norm=asinh)

        # Build a model for each component in the model
        rgb = []
        for _model in model:
            # Convert the model to an RGB image
            _rgb = scarlet.display.img_to_rgb(_model, filter_indices=[3,2,1], norm=asinh)
            rgb.append(_rgb)

        # Display the image and model
        figsize = [6,3]
        columns = 2
        # Calculate the number of columns needed and shape of the figure
        if show_sed:
            figsize[0] += 3
            columns += 1
        if not combine:
            figsize[0] += 3*(model.shape[0]-1)
            columns += model.shape[0]-1
        # Build the figure
        fig = plt.figure(figsize=figsize)
        ax = [fig.add_subplot(1,columns,n+1) for n in range(columns)]
        ax[0].imshow(img_rgb)
        ax[0].set_title("Data")
        for n, _rgb in enumerate(rgb):
            ax[n+1].imshow(_rgb)
            if combine:
                ax[n+1].set_title("Initial Model")
            else:
                ax[n+1].set_title("Component {0}".format(n))
        if show_sed:
            for sed in src.sed:
                ax[-1].plot(sed)
            ax[-1].set_title("SED")
            ax[-1].set_xlabel("Band")
            ax[-1].set_ylabel("Intensity")
        # Mark the current source in the image
        y,x = src.center
        ax[0].plot(x-src.bb[2].start, y-src.bb[1].start, 'wx', mew=2)
        plt.tight_layout()
        plt.show()

## Introduction

The purpose of this overview is to understand the basic structure of the *scarlet* package, present some of the most useful parameters for customizing scenes, and explain how to extend the use of *scarlet* for more specialized science cases.
The [User Documentation](user_docs.rst) contains more detailed descriptions of the modules and classes used in *scarlet*, and a more rigorous overview of the mathematics and algorithms used by *scarlet* is described in [Moolekamp and Melchior 2018](https://arxiv.org/abs/1708.09066) and [Melchior et al. 2018](https://arxiv.org/abs/1802.10157).
In the future we also expect to add a tutorial section to give examples of more specialized science cases.

### Basic Structure

*scarlet* is designed to separate sources in astrophysical images by assuming that each scene (or [Blend](blend.ipynb#scarlet.blend.Blend)) can be thought of as a collection of mutiple [Source](source.ipynb#scarlet.source.Source) objects.
The [Blend](blend.ipynb#scarlet.blend.Blend) class contains all of the information about the scene, including  routines for fitting sources and generating models.
Each [Source](source.ipynb#scarlet.source.Source) in a [Blend](blend.ipynb#scarlet.blend.Blend) can be made up of multiple components, where each component has a single morphology (shape) and uniform spectrum (or SED) with a set of [Constraint](constraints.ipynb#scarlet.constraints.Constraint) objects used to break degeneracies in the model.


### Object Detection

At present *scarlet* does not have the functionality to detect sources, so in addition to the original blended image, *scarlet* requires a catalog of source coordinates in the image.
As we will see later, a pitfall of the internal fitting algorithms used by the deblender is that they are very sensitive to undetected sources.
This means that complicated blends may require iteratively detecting and modeling a scene multiple times to ensure that all of the sources have been accounted for.
It is possible that a future version of *scarlet* might include multiband detection, but for now the user must use a package like [SEP](http://sep.readthedocs.io/) or [photutils](https://photutils.readthedocs.io/en/stable/) to perform source detection.

## [Sources](source.ipynb#scarlet.source.Source)

The base [Source](#scarlet.source.Source) class requires an initial guess for the SED and morphology of each source (or a list of seds and morphologies if a source has mutliple components, eg. a bulge and a disk).
For most general use cases neither the SED nor the morphology are known ahead of time, so a set of convenience classes, [PointSource](source.ipynb#scarlet.source.PointSource) and [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource), exist to generate an initial morpholgy and SED for a source at a given location in the image.
It is also possible for users to specify their own source types with custom initialization methods.

### Initialization
The simplest [Source](#scarlet.source.Source) can be created using


In [ ]:
B, Ny, Nx = images.shape
center = (Ny//2, Nx//2)
# Get the SED at the location of the central pixel
sed = np.expand_dims(scarlet.source.get_pixel_sed(images, center), axis=0)
# Set the morphology such that only the central pixel has any intensity
morph = np.zeros((Ny, Nx))
morph[center] = 1
src = scarlet.Source(sed=sed, morph_image=np.expand_dims(morph, axis=0))

Notice that while sources will typically have a `center` (position) argument specified, more exotic objects like jets or optical artifacts may not have a well defined center, so the `center` (coordinates) of a source in the image is not required for a general [Source](#scarlet.source.Source) (however both [PointSource](source.ipynb#scarlet.source.PointSource) and [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) do require a center during initialization).
Also note that the center required by *scarlet* is the *geometric centroid* of the source, not the flux weighted average position that (unfortunately) is misleadingly often refered to as the "centroid" in astronomy.

In general, most sources will fit in some subspace of the full image, so it is useful to initialize the source in the smallest possible frame (or bounding box).
When the entire blend is fit, the size of the frame is recalculated if `fix_frame` is `False` (the default value).
However, setting `fix_frame` to `True` will prevent the source from growing in size, which may be desireable in certain use cases.

Due to the nature of the symmetry and monotonicity constraints to be examined later, it is useful to recenter all of the sources periodically to null dipoles that occur when objects are even slightly mis-aligned.
If there is a good reason to believe that the initial positions are correct (such as in simulations) or that fitting them will create undesirable behavior, it is possible to set `shift_center=0`, which will cause the position of the current source to remain fixed.


While the [Source](#scarlet.source.Source) base class can be used to initialize a source, it is more common to initialize a source using an inherited class.
For example, [PointSource](source.ipynb#scarlet.source.PointSource) creates a new source given a `center` position using the SED of the pixel at that location in each band with only that single pixel turned on in the morphology.
A more useful initialization class is the [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource), which initializes a source that is symmetric and monotonically decreasing about the peak `center` location.
For example, to create a list of sources for every source in the input catalog:

In [ ]:
sources = [scarlet.source.ExtendedSource((src['y'],src['x']), images, bg_rms) for src in catalog]

Notice that [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) has a few additional arguments to the [Source](#scarlet.source.Source) base class.
It requires `img`, the image datacube used to initialize the SED and morphology, and the background level `bg_rms`.
The background level is needed so that the frame (or bounding box) for the source can be made as compact as possible by trimming the box at the `bg_rms` noise level during initialization (however unless `fix_frame` is turned off, the box can be resized later if necessary.

For creating your own custom source initialization methods, see [Custom Sources](#Custom-Sources).

## Constraints

### Introduction to constraints in *scarlet*
Part of the utility of *scarlet* comes from it's ability to allow users to create custom constraints.
A thorough mathematical and algorithmic explanation of how constraints are applied to sources in the model can be found in [Moolekamp and Melchior 2018](https://arxiv.org/abs/1708.09066) and [Melchior et al. 2018](https://arxiv.org/abs/1802.10157).
What follows is a description of the data structures in *scarlet* needed to apply those constraints to the model and how they can be extended or customized by an end user, however the user is referred to those papers for an explanation of the notation.

As described in the paper mentioned above, to allow both smooth and non-smooth (non-differentiable) constraints the underlying algorithm in *scarlet* uses proximal operators, which can be as simple as projections onto the constrained space or more complicated functions (for a good reference on proximal operators see [Parikh and Boyd 2014](http://www.web.stanford.edu/~boyd/papers/pdf/prox_algs.pdf) and the appendix in [Combettes and Pesquet 2011](https://link.springer.com/chapter/10.1007/978-1-4419-9569-8_10)).
*scarlet* allows for two different kinds of constraints: proximal operators that act on the likelihood ($prox_f(A)$ or $prox_f(S)$) and constraints that act on a different vector space accessed through a linear operator ($prox_g \left(\mathsf{L} \cdot A_k \right)$ or $prox_g \left(\mathsf{L} \cdot S_k \right)$.
$prox_f$ constraints can be thought of as *strict* constraints, in that each step of the fitting algorithm will result in an update to the variable that obeys the constraint.
Conversely, $prox_g$ constraints do not act on the variable directly and instead will converge to a solution that obeys the constraint, meaning intermediate steps might violate one or more of the constraints.
Users should take this behavior into consideration before deciding which constraint to use.

For example, the [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) is (by default) a symmetric and monotonic model.
It has been our experience that using a prox$_g$ monotonicity constraint does not work well, as monotonicity is basically a radial gradient and if a single pixel disobeys the monotonicity constraint, pixels radially further from the peak can also be unnaturally high as they are technically monotonicaly decreasing from the single hot pixel.
Instead, [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) implements a strict monotonicity projection that guarantees that the model is monotonic in every step.
On the other hand, because most galaxies are not perfectly symmetric, it can be beneficial to have a symmetry constraint that is not strictly enforced in each step, so by default [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) has a prox$_g$ symmetry constraint.

Constraints are added to individual components of sources via the [Constraint](constraints.ipynb#scarlet.constraints.Constraint) base class.
A single [Constraint](constraints.ipynb#scarlet.constraints.Constraint) object can have mutliple different constraints, combined in a [ConstraintList](constraints.ipynb#scarlet.constraints.ConstraintList).

For example, the [MinimalConstraint](constraints.ipynb#scarlet.constraints.MinimalConstraint), which requires the SED to have non-negative elements that sum to unity and the morphology to be non-negative, can be combined with an [L0Constraint](constraints.ipynb#scarlet.constraints.L0Constraint) using

In [ ]:
constraint = sc.MinimalConstraint() & sc.L0Constraint(0.1)
constraint

We can look at the $prox_f$ constraints for the sed:

In [ ]:
constraint.prox_sed

which shows (as expected) that the sed must be positive and sum to unity.

We also see that there are no prox_g constraints or linear operators:

In [ ]:
print(constraint.prox_g_sed)
print(constraint.prox_g_morph)
print(constraint.L_sed)
print(constraint.L_morph)

But we get a surprise when we look at $prox_f$ for the morphology:

In [ ]:
constraint.prox_morph

This is because we have applied two different $prox_f$ constraints by "daisy chaining" them together.
Because the different $prox_f$ constraints are not guaranteed to commute, the `proxmin.operators.AlternatingProjections` object alternates the order it applies each proximal operator or projection to make the solution more robust.
Examining this object shows the two proximal operators it alternates between:

In [ ]:
constraint.prox_morph.operators

We can (in theory) combine as many constraints as we want:

In [ ]:
constraint = constraint & sc.PositivityConstraint()

In this case, because the [PositivityConstraint](constraints.ipynb#scarlet.constraints.PositivityConstraint) applies both [prox_center_on](operators.ipynb#scarlet.operators.prox_center_on) (which forces the center pixel to have a small but non-zero value to prevent the recentering algorithm from crashing) and `proxmin.operators.prox_plus`, we get `proxmin.operators.prox_plus` twice, which is a waste of CPU time:

In [ ]:
constraint.prox_morph.operators

A better way to achieve the same result would be to use

In [ ]:
constraint = sc.SimpleConstraint() & sc.L0Constraint(0.1)

where [SimpleConstraint](constraints.ipynb#scarlet.constraints.SimpleConstraint) inherits from [PositivityConstraint](constraints.ipynb#scarlet.constraints.PositivityConstraint) and adds `proxmin.prox_unit_plus` to `prox_sed`.
So the above command adds [L0Constraint](constraints.ipynb#scarlet.constraints.L0Constraint) to `prox_morph` :

In [ ]:
print(constraint.prox_sed)
constraint.prox_morph.operators

### Symmetry

Approximating astrophysical sources as symmetric objects was a clever insight made by the SDSS deblender, which is also used in the HSC and LSST software stacks until an optimized version of *scarlet* is implemented.
The idea is that when neighboring objects are blended together, the flux (or lack thereof) on the side opposite the neighbor can be used to force it's symmetric partner to a lower flux value.
In the case of the SDSS-HSC deblender this template is taken as an approximation to the actual morphology and the original image is re-weighted using the symmetrized templates.

*scarlet* takes the idea of symmetry more seriously, as the templates generated for each sorce are considered *the* model of the object.
In this regime symmetry is forced on the object, even in cases where a source might not be exactly symmetric (for example grand design spirals, irregular galaxies, or jets).
We are currently working on an implementation that imposes a less strict penalty for breaking symmetry, but what follows is the current symmetry constraint in *scarlet*.

Symmetry is implement as a $prox_g$ proximal operator, where a linear matrix $L=$[getSymmetryOp](transformations.ipynb#scarlet.transformations.getSymmetryOp) takes the difference between each pixel and it's symmetric partner and the proximal operator `proxmin.operators.prox_zero` forces the dual variable onto the perfectly symmetric space (where all symmetric pair differences are zero).
As stated in the previous subsection, this constraint is not strictly met in any given iteration but will converge to a solution where the morphology is nearly symmetric.

We can add this constraint to our previous `constraint` using

In [ ]:
constraint = constraint & sc.SymmetryConstraint()

Now we see that our constraint has a `prox_g_morph` but still no `L_morph`

In [ ]:
print(constraint.prox_g_morph)
print(constraint.L_morph)

This is because the linear operator can't be built until the source is initialized (because it needs to know the shape of the morphology), which happens when a new [Source](#scarlet.source.Source) is created by calling [Source.set_constraints()](source.ipynb#scarlet.source.Source).
If we initialize a source with this constraint:

In [ ]:
src = scarlet.source.PointSource((catalog["y"][0], catalog["x"][0]),
                                 img=images, shape=(15, 15),
                                 constraints=constraint)
print(constraint.L_morph)

we see that the linear operator has been created.

### Monotonicity

Another key insight of the SDSS-HSC deblender is the approximation that most astrophysical objects are roughly monotonically decreasing from the peak.
Of course this is also violated in spiral galxies, especially ones that are tightly wound.
But if we think of spirals as a single source made up of multiple components, each monotonically decreasing from it's peak with a single SED, we can build a model that is well representative of the full galaxy.
This point of view has the added benefit that regions that are not monotonically decreasing in a galaxy are likely different stellar populations with (potentially) different SED's and should be treated as separate components anyway.

Monotonicity in *scarlet* is implemented as both a $prox_f$ and $prox_g$ constraint, so we will look at each use to see their differences.
The $prox_g$ implementation uses the [MonotonicityConstraint](constraints.ipynb#scarlet.constraints.MonotonicityConstraint) class to build a [getRadialMonotonicOp](transformations.ipynb#scarlet.transformations.getRadialMonotonicOp) linear operator that takes the differences between the flux in the reference pixels and the flux in the current pixel.
If `use_nearest` is `True`, only a single reference pixel is used, the pixel that is nearest to the current pixel in line with the peak. Otherwise a weighted average of all pixels closer to the peak than the current pixel is used to allow for a smoother monotonic solution.

| ![](images/nearest_ref.png) | ![](images/weighted_ref.png) |
|:---------------------------:|:----------------------------:|
| Nearest Neighbor            | Weighted Reference           |

For the $prox_g$ implementation, the `proxmin.operators.prox_plus` proximal operator is used to project the result of the linear operator onto the space where all values are non-negative, ie. the monotonic space.
But because this is not a strict operator, a single "hot" pixel will cast a shadow into the noise as all of the pixels using it as a reference are technically monotonic.

In the following example, for simplicity we use the [PointSource](source.ipynb#scarlet.source.PointSource) class to create a source with a simple initial SED and morphology, but add both a [SymmetryConstraint](constraints.ipynb#scarlet.constraints.SymmetryConstraint) and a [MonotonicityConstraint](constraints.ipynb#scarlet.constraints.MonotonicityConstraint) to allow the model to converge to a monotonic solution.

In [ ]:
# Initialize the Sources
sources = [scarlet.source.PointSource(
    (src['y'],src['x']), # center coordinates in `images`
    images, # data cube (bands, Ny, Nx)
    (15,15), # initial shape of the bounding box
    constraints=(
        sc.SimpleConstraint() # sed sum to unity, all elements of SED and morph are non-negative
        & sc.MonotonicityConstraint(use_nearest=True) # prox_g monotonicity
        & sc.SymmetryConstraint() # prox_g symmetry
    ),
) for src in catalog]

Now we create a blended scene with all of the sources.
We'll discuss the [Blend](blend.ipynb#scarlet.blend.Blend) class more later, but for now we just use it to run a few iterations to see $prox_g$ monotonicity in action.

In [ ]:
blend = scarlet.Blend(sources, images, bg_rms=bg_rms)
blend.fit(50)
# For simplicity only show the first 3 peaks
display_sources(sources, subset=[0,1,2])

While sources 1 and 2 look ok, we see that source 0 is clearly not monotonically decreasing from the center (although if you look closely neither is source 1).

Now we'll use the direct $prox_f$ monotonicity, which is written in C++ and forces all of the pixels to be monotonically decreasing by starting at the center pixel and working radially outward to enforce monotonicity. This also comes in a weighted and nearest neighbor version, so for consistency we use the nearest neighbor monototonicity again:

In [ ]:
# Initialize the Sources
sources = [scarlet.source.PointSource(
    (src['y'],src['x']), # center coordinates in `images`
    images, # data cube (bands, Ny, Nx)
    (15,15), # initial shape of the bounding box
    constraints=(
        sc.SimpleConstraint() # sed sum to unity, all elements of SED and morph are non-negative
        & sc.DirectMonotonicityConstraint(use_nearest=True) # prox_f monotonicity
        & sc.SymmetryConstraint() # prox_g symmetry
    ),
) for src in catalog]

# Create the blend and display the sources
blend = scarlet.Blend(sources, images, bg_rms=bg_rms)
blend.fit(50)
display_sources(sources, subset=[0,1,2])

So we see significant improvement using the direct monotonicity, even though (for reasons beyond the scope of this document) it is not an exact proximal operator.

### Custom Sources

Many users may require specialized features that are not implemented in the simple [Source](#scarlet.source.Source) classes defined in *scarlet*, so the framework is built to support different initialization schemes in the form of inherited classes.
For example, the following code creates a simple bulge-disk model:

In [ ]:
class BulgeDisk(scarlet.Source):
    """A galactic source with two components
    """
    def __init__(self, center, img, config=None):
        self.center = center
        if config is None:
            config = Config()
        # Use the smallest cache size for the initial bounding box
        shape = (config.source_sizes[0],) * 2
        # Use the same constraints as the default `ExtendedSource`
        constraints = (sc.SimpleConstraint() &
                       sc.DirectMonotonicityConstraint(use_nearest=False) &
                       sc.SymmetryConstraint())
        # Initialize the SEDs and morphologies for both components
        sed, morph = self.make_initial(img, shape)
        # WARNING: Don't forget to initialize the `Source`, as there is a lot
        # of internal initialization
        super(BulgeDisk, self).__init__(sed, morph, center=center, constraints=constraints)

    def make_initial(self, img, shape):
        B, Ny, Nx = img.shape
        # A small but non-zero number
        tiny = 1e-10
        # center_int is a property of a `Source` that returns the
        # integer position of the coordinates
        _y, _x = self.center_int

        # Get the color of the center pixel
        disk_sed = scarlet.source.get_pixel_sed(img, self.center_int)
        # Turn on all of the pixels in the box for the disk
        disk_morph = np.zeros(shape)
        cy = shape[0] >> 1
        cx = shape[1] >> 1
        dy = shape[0] >> 2
        dx = shape[1] >> 2
        disk_morph[cy-dy:cy+dy+1, cx-dx:cx+dx+1] = max(img[:,_y,_x].sum(axis=0), tiny)
        disk_morph = disk_morph.reshape(shape[0]*shape[1])

        # Make the bulge SED redder (and normalize)
        bulge_sed = disk_sed + np.linspace(-0.1, 0.1, B)
        bulge_sed /= np.sum(bulge_sed)
        # Turn on a single pixel for the bulge
        bulge_morph = np.zeros(shape[0] * shape[1])
        center_pix = bulge_morph.size // 2
        bulge_morph[center_pix] = max(img[:,_y,_x].sum(axis=0), tiny)

        # Combine the two components into an initial `sed` and `morph`
        sed = np.array([bulge_sed, disk_sed])
        morph = np.array([bulge_morph, disk_morph]).reshape(2, shape[0], shape[1])
        return sed, morph

In this simple example a new bulge-disk model is initialized with only the central coordinates of the source and the image of the scene.
The source is initialized with the smallest available source size (see [Configuration](#Configuration)) and uses the same constraints as an [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource), namely symmetry, monotonicity, and an SED that sums to unity (see [Constraints](#Constraints)).
The initial SED and morphology are defined in the `make_initial` method, which initializes the morphology with a disk component that is half the size of the initial bounding box and a bulge component, which is a single pixel slightly redder than the disk.
Finally the parent [Source](source.ipynb#scarlet.source.Source) class is initialized.

We can now create a list of sources with two components, initialized using the `BulgeDisk` class above.

In [ ]:
bd_sources = [BulgeDisk((src["y"], src["x"]), images) for src in catalog]
display_sources(bd_sources, range(3))

If we run *scarlet* for a few iterations we see that the two components have begun to converge toward two separate solutions, although a second component doesn't appear to be necessary for any of these objects.

In [ ]:
bd_sources = [BulgeDisk((src["y"], src["x"]), images) for src in catalog]
blend = scarlet.Blend(bd_sources, images, bg_rms=bg_rms)
blend.fit(100)
display_sources(blend.sources, range(3))

## Blended Scenes

The [Blend](blend.ipynb#scarlet.blend.Blend) class contains the entire blended scence.

### Configuration

A number of global configuration options, used by both [Blend](blend.ipynb#scarlet.blend.Blend) and [Source](#scarlet.source.Source) objects, are accessed through the [Config](config.ipynb#scarlet.config.Config) class, making it easier to pass configuration options between a [Blend](blend.ipynb#scarlet.blend.Blend) and it's sources.
See [Configuration](config.ipynb#Configuration-(scarlet.config)) for a detailed description of different configuration options.

Most of the properties can be modified by the user on the fly, but changing the `Config.source_sizes` propery should be accomplished by using the [Config.set_source_sizes](config.ipynb#scarlet.config.Config.set_source_sizes) method, which ensures that all of the `source_sizes` are valid (see [Config](config.ipynb#scarlet.config.Config) for more).

### Initialization

Initializing a new blended scene requires a list of `sources` ([Source](#scarlet.source.Source) objects), an image (`img`) datacube with dimensions (`bands`, `height`, `width`).
If a weightmap exists, the user can also pass a `weights` datacube with the same dimensions as `img`.
It is also useful to pass a value for the background RMS (`bg_rms`), which is used to minimize the box size needed for each source (see the discussion in [Resizing Sources](#Resizing-Sources)).
Finally an optional [Config](config.ipynb#scarlet.config.Config) class can be specified, with custom configuration options.
If no `config` is given, a new [Config](config.ipynb#scarlet.config.Config) instance is created using the default values.

For most users, a good place to start is by defining each source as an [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) and initializing a blend with

In [ ]:
sources = [scarlet.source.ExtendedSource((src['y'],src['x']), images, bg_rms) for src in catalog]
blend = scarlet.Blend(sources, images, bg_rms=bg_rms)

This creates a scene with a collection of sources, each one with a single SED and a morphology that is both monotonic and symmetric (see [Sources](#Sources)), and will use `bg_rms` to minimize the box sizes of each source.

To customize the configuration of the [Blend](blend.ipynb#scarlet.blend.Blend), for example using a lower flux threshold for resizing source bounding boxes, a [Blend](blend.ipynb#scarlet.blend.Blend) can be initialized with a new [Config](config.ipynb#scarlet.config.Config).

In [ ]:
config = Config(edge_flux_thresh=0.5)

which will resize the box as long as flux greater than `Config.edge_flux_thresh*bg_rms`.

### Accessing Individual Sources and Components

The bSDMM algorithm implemented by the [proxmin](https://github.com/pmelchior/proxmin) package expects A and S to be 2D matrices, however duck typed class are used to make more complicated tensors act as if they were 2D matrices.
Internally the algorithm operates on the space of components, not sources (where a single [Source](#scarlet.source.Source) might have multiple components), so a [Blend](blend.ipynb#scarlet.blend.Blend) contains a number of indexing properties to convert from component space to source space.

For example, a `blend` created using the `BulgeDisk` class has two components for each source, for a total of 2$\times$sources components.

In [ ]:
bd_sources = [BulgeDisk((src["y"], src["x"]), images) for src in catalog]
blend = scarlet.Blend(bd_sources, images, bg_rms=bg_rms)
print("Number of sources:", len(blend.sources))
print("Number of components:", blend.K)

If we iterate through the sources we can find the index of the component in `blend`:

In [ ]:
for m, src in enumerate(blend.sources):
    for l in range(src.K):
        k = blend.component_of(m,l)
        print("Component {0} has index {1}".format((m,l),k))

Similarly, we can use the index of the model component $k$ to find the index of the source, and the index of the component inside that source:

In [ ]:
for k in range(blend.K):
    m,l = blend.source_of(k)
    print("Component {0} is component {1} in source {2}".format(k, l, m))

[Blend](blend.ipynb#scarlet.blend.Blend) also has a `__len__` method, so taking the length of `blend` gives the number of sources (not components):

In [ ]:
len(blend)

### Fitting a Model

Most of the internal methods of the [Blend](blend.ipynb#scarlet.blend.Blend) class are used to implement the minimization algorithm described in [Moolekamp and Melchior 2018](https://arxiv.org/abs/1708.09066).
Below is a very brief summary of the method to elucidate how constraints are implemented in *scarlet*.

The basic deblending algorithm builds a model

$$M= \sum_{k=1}^K A_k^T \times S_k = AS, $$

where $A_k \in \mathbb{R}^B$ is the normalized SED and $S_k \in \mathbb{R}^N$ is the morphology of a single component in the model with $B$ bands and $N$ pixels in each band.

The blend is fit by minimizing the likelihood of the model, namely minimizing

$$f(A,S) = \frac{1}{2} || Y-AS ||_2^2, $$

where $Y$ is the data (image) and $||.||_2$ is the element-wise $L2$ (Frobenius) norm.

Constraints are applied to each source in the form of proximal operators, a handy mathematical tool for imposing non-smooth constraints that (if properly formulated) are guaranteed to converge.
Each component $j$ can have multiple constraints $M_j$, this is equivilent to minimizing

$$f(A, S) + \sum_{j=1}^K \sum_{i=1}^{M_j} g_{ji} \left(Z_{ji} \right)$$

subject to

$$L_{ji} S_j - Z_{ji} = 0 \ \forall\ j \in \{1, \dotso, K \} \textrm{and } i\in \{1, \dotso, M_j\},$$

where $K$ is the number of components, $g_{ji}$ is a constraint and $L_{ji}$ is a linear operator for the $i$th constraint of the $j$th component, and $Z_{ji}$ is a dual variable used to apply the constraint using the Block-Simultaneous Method of Multipliers (bSDMM, [Moolekamp and Melchior 2018](https://arxiv.org/abs/1708.09066)).

While the details of the algorithm are not important for using *scarlet*, users interested in customizing the deblender for specific science cases will find it helpful to understand how $A$ and $S$ are updated, namely:

$$ x^{\textrm{it}+1} \leftarrow \textrm{prox}_{\lambda f} \left( x^{\textrm{it}} - \frac{\lambda}{\rho} \mathsf{L}^T \left( \mathsf{L} x^{\textrm{it}}-z^{\textrm{it}} + u^{\textrm{it}} \right) \right)$$

$$z^{\textrm{it}+1} \leftarrow \textrm{prox}_{\rho g} \left( \mathsf{L} x^{\textrm{it}+1} + u^{\textrm{it}} \right)$$

$$u^{\textrm{it}+1} \leftarrow u^{\textrm{it}} + \mathsf{L} x^{\textrm{it}+1} - z^{\textrm{it}+1}$$

where $x^{\textrm{it}}$ is either $A_k$ or $S_k$ in the current iteration, $\lambda$ and $\rho$ are step sizes for $\textrm{prox}_f$ and $\textrm{prox}_g$, $z^{\textrm{it}}$ is the dual variable in the current iteration, and $u^{\textrm{it}}$ is the running difference between the constrained and current values of $\mathsf{L} x$.

The [Blend.fit](blend.ipynb#scarlet.blend.Blend.fit) method is used to fit the current model and requires only two parameters: the maximum number of `steps` (or iterations) used to fit the data and the relative error for convergence (`e_rel`).
[Blend.fit](blend.ipynb#scarlet.blend.Blend.fit) builds the appropriate objects and calls `proxmin.algorithms.bsdmm` to fit the data, which will run until one of the following conditions is met:

1. The total number of iterations is equal to `steps`

1. The model converges (as defined in  [Moolekamp and Melchior 2018](https://arxiv.org/abs/1708.09066)).

1. The algorithm throws a [ScarletRestartException](blend.ipynb#scarlet.blend.ScarletRestartException). This occurs when some variable is modified that is incompatible with continued execuation of `proxmin.algorithms.bsdmm`, usually the size or shape of at least one [Source](source.ipynb#scarlet.source.Source).

1. The algorithm crashes for some other uncaught reason. We have attempted to generate useful error messages for common failure modes but new users are likely to find edge cases that we have not yet caught.

If `proxmin.algorithms.bsdmm` exists due to a [ScarletRestartException](blend.ipynb#scarlet.blend.ScarletRestartException) and the total number of iterations (`it`) is less than `steps`, the sources are re-initialized and `proxmin.algorithms.bsdmm` is executed for `steps-it` iterations.
In other words, even if the algorithm has to restart, [Blend.fit](blend.ipynb#scarlet.blend.Blend.fit) will never run for more than `steps`.

### Restarting a Fit

There may be instances where it is desirable to restart a fit.
For example, after a certain number of iterations you might want to add or remove new sources, or you may have a custom constraint that you want to apply every Nth iteration.
In that case you can call `Blend.fit(N)`, perform your cut, and restart for another `Blend.fit(N)`:

In [ ]:
sources = [scarlet.source.ExtendedSource((src['y'],src['x']), images, bg_rms) for src in catalog]
blend = scarlet.Blend(sources, images, bg_rms=bg_rms)
blend.fit(20)
print(blend.it)
blend.fit(20)
print(blend.it)

Notice that blend is now on iteration `2*N`, with `N=20` in the example above.

### Resizing Sources

In most blends the number of non-zero pixels is $\ll$ than the total number of pixels in the image.
To save processing time we recommend initializing sources with a minimal number of pixels and allowing the [Blend.fit](blend.ipynb#scarlet.blend.Blend.fit) method to expand the size of the box if necessary by calling the [Blend.resize_sources](blend.ipynb#scarlet.blend.Blend.resize_sources) method.
This method uses the update for each source's morphology to determine if the bounding box for a source needs to be increased (which happens when the morphology has any flux above `config.edge_flux_thresh*blend.bg_rms` outside it's current bounding box).

### Recentering Sources

Due to the default [symmetry](#Symmetry) and [monotonicity](#Monotonicity) constraints, the model is dependent on the accuracy of the center position for each source.
If the center is off by a fraction of a pixel then both monontonicy and symmetry force the model away from the data and can cause large residuals in the resulting model.
The [Blend.recenter_sources](blend.ipynb#scarlet.blend.Blend.recenter_sources) method recenters all of the sources simultaneously by constructing a difference image, a copy of the current model shifted by an additional `Source.shift_center` pixels (usually a small number around 0.2).
The function then simultaneously calculates the shift needed to align the models with the data to null any positional dipoles.

Both resizing sources and recentering sources are expensive operations (as resizing requires rebuilding all of the constraint operators and recentering requires inverting all of the models in the original image) and are only performed every `Config.refine_skip` iterations.

## Loading and Displaying a Model

### Display functions

The [display](display.ipynb) module contains a number of convenience methods to convert an image cube into an RGB image array.

There are two stock classes used to scale the pixels in the image, [Linear](display.ipynb#scarlet.display.Linear) and [Asinh](display.ipynb#scarlet.display.Asinh), both of which inherit from the `matplotlib.colors.Normalize` class.
This inheritance allows them to be used as normalizations in `matplotlib.pyplot.imshow`, including an `inverse` method that makes it possible to add a colorbar.
For example:

In [ ]:
norm = scarlet.display.Linear(img=images)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Linear Scaling")
plt.show()

norm = scarlet.display.Asinh(img=images)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Asinh Scaling")
plt.show()

The [Linear](display.ipynb#scarlet.display.Linear) class is fairly straightforward, allowing the user to (optionally) pass `vmin` and `vmax` parameters to set the range of the data:

In [ ]:
norm = scarlet.display.Linear(vmin=0, vmax=100)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Linear Scaling")
plt.show()

The [Asinh](display.ipynb#scarlet.display.Asinh) scaling is popular because it is linear for small values and logrithmic for larger fluxes, allowing it to display a wide range of intensities clearly.
The actual formula used to scale each pixel is

$$f(x) = \frac{1}{Q} \sinh^{-1} \left( Q \frac{x-v_\textrm{min}}{v_\textrm{max}-v_\textrm{min}} \right)$$

where `Q` is a parameter that defines the strech of the scaling.

In [ ]:
norm = scarlet.display.Asinh(img=images, vmin=0, Q=10)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Asinh Scaling")
plt.show()

norm = scarlet.display.Asinh(img=images, vmin=0, Q=100)
plt.imshow(images[2], norm=norm)
plt.colorbar()
plt.title("Asinh Scaling")
plt.show()

An image cube can be converted from a (bands, height, width) array into an RGB image array using the [img_to_rgb](display.ipynb#scarlet.display.img_to_rgb) function.
This allows the user to specify a normalization (`norm`), `fill_value` (value to use for any masked pixels), and list of indices to map to R, G, B respectively (`filter_indices`).
For example the default is to map the first three bands in reverse order to RGB:

In [ ]:
img_rgb = scarlet.display.img_to_rgb(images, norm=asinh)
plt.imshow(img_rgb)

In this case the mapping is $r \rightarrow R$, $g \rightarrow G$, $u \rightarrow B$.
A more natural mapping (and the one used in most of this document) is $i \rightarrow R$, $r \rightarrow G$, $g \rightarrow B$:

In [ ]:
img_rgb = scarlet.display.img_to_rgb(images, filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)

### Extracting Models from a Blend

You can load the models that make up a scene at any time, even if the blend has been initialized but not fit for a single iteration. For example:

In [ ]:
# Initialize the blend but don't fit the model
bd_sources = [BulgeDisk((src["y"], src["x"]), images) for src in catalog]
blend = scarlet.Blend(bd_sources, images, bg_rms=bg_rms)
model = blend.get_model()
img_rgb = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)

It is also possible to load the model for each source individually:

In [ ]:
models = blend.get_model(
    combine=False, # Don't combine the model for each source together
    flat=False # Don't flatten the model, keeping each component separate
)
img_rgb = scarlet.display.img_to_rgb(models[0], filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)
plt.show()

or even by component:

In [ ]:
# Load a model 
model = blend.get_model(
    m=0, # index of the first source
    combine=False, # Don't combine the model for each source together
    combine_source_components=False # Don't combine the components into a single model for each source
)
# Display the bulge
img_rgb = scarlet.display.img_to_rgb(model[0], filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)
plt.show()
# Display the disk
img_rgb = scarlet.display.img_to_rgb(model[1], filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)
plt.show()

You might also want to extract just the morphology (without convolving with the SED). To prevent unexpected results (i.e. the model having a different shape depending on the given parameters) the resulting model will still have the shape (`components`, `bands`, `height`, `width`), however there are only `components` different results.

In [ ]:
# Run a few iterations, jsut to make the model more interesting
blend.fit(100)
model = blend.get_model(
    m=0, # index of the first source
    combine=False, # Don't combine the model for each source together
    combine_source_components=False, # Don't combine the components into a single model for each source
    use_sed=False # Don't convolve with the SED
)

In [ ]:
plt.imshow(model[0][1])
plt.show()
plt.imshow(model[0][3])
plt.show()
plt.imshow(model[1][1])
plt.show()
plt.imshow(model[1][2])
plt.show()

### Extracting Models from a Source

It is also possible to access the morphology of a given source directly, which is always centered:

In [ ]:
src = blend.sources[0]
plt.imshow(src.image[0])
plt.show()

plt.imshow(src.image[1])
plt.show()

where `src.image` is the same as `src.morph.reshape(src.K, src.Ny, src.Nx)`.
Notice how in this space only the part of the model inside the bounding box is given, since this is the space where the morphology lives.

We can also look at the SED for each source:

In [ ]:
for m, src in enumerate(blend.sources):
    # Only display the SED for the bulge components
    plt.plot(src.sed[0], label=m)
plt.legend()

It is also possible to extract the model for a given source:

In [ ]:
model = src.get_model()
_model = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=asinh)
plt.imshow(_model)

or by component:

In [ ]:
model = src.get_model(combine=False)
_model = scarlet.display.img_to_rgb(model[0], filter_indices=[3,2,1], norm=asinh)
plt.imshow(_model)

or without recentering  or PSF convolution:

In [ ]:
Gamma = src._gammaOp([0,0], src.shape)
model = src.get_model(Gamma=Gamma)
_model = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=asinh)
plt.imshow(_model)

This works because the `Source._gammaOp` is an operator that handles psf convolution and translation. This part of the code is likely to change in the near future, so we won't give a detailed description of how this is implementd in the code. But basically, `Source._gammaOp([dy,dx])` performs a psf convolution (if a psf was specified) and a linear shift by $dx$ and $dy$, the fraction of a pixel to move in the $x$ and $y$ directions respectively.

## Other Useful Source Properties

Below are a list of properties that can be accessed for a source. For more information about them, see [Source](source.ipynb#scarlet.source.Source).

In [ ]:
src = blend.sources[1]
print("Source shape:", src.shape)
print("Source dimensions:", (src.Ny, src.Nx))
print("Center:", src.center)
print("integer center:", src.center_int)
print("Slice of the original image to fit the source:", src.get_slice_for(images.shape))

Display the image centered on the source, where `Source.bb` is the bounding box to extract the source:

In [ ]:
_img = images[src.bb]
img_rgb = scarlet.display.img_to_rgb(_img, filter_indices=[3,2,1], norm=asinh)
plt.imshow(img_rgb)

If a source is on the edge then it might be necessary to extract only part of the full source using [Source.get_slice_for](source.ipynb#scarlet.source.Source.get_slice_for), where `source.image[source.get_slice_for(images.shape)]` is the same shape and aligned with `images[source.bb]`:

In [ ]:
fig = plt.figure(figsize=(6,3))
ax = [fig.add_subplot(1,2,1+n) for n in range(2)]
ax[0].imshow(img_rgb)
_img = scarlet.display.img_to_rgb(src.get_model()[src.get_slice_for(images.shape)], filter_indices=[3,2,1], norm=asinh)
ax[1].imshow(_img)

This concludes the overview. The user is referred to the [User Documentation](user_docs.rst) for more details about the objects used in *scarlet*.